In [6]:
from openmmtools.integrators import LangevinIntegrator
from simtk import openmm
from simtk.openmm import unit, app
import numpy as np
from openmmforcefields.generators import SystemGenerator

In [3]:
pdb = app.PDBFile("../../input/N501Y_capped_peptide.pdb")

In [7]:
# Generate system
forcefield_files = ['gaff.xml', 'amber14/protein.ff14SB.xml', 'amber14/tip3p.xml']
barostat = None
system_generator = SystemGenerator(forcefield_files,
                               barostat=barostat,
                               forcefield_kwargs={'removeCMMotion': False,
                                                    'ewaldErrorTolerance': 1e-4,
                                                    'constraints' : app.HBonds,
                                                    'hydrogenMass' : 4 * unit.amus},
                                nonperiodic_forcefield_kwargs={'nonbondedMethod': app.NoCutoff},
                                small_molecule_forcefield='gaff-2.11',
                                molecules=None,
                                cache=None)
system = system_generator.create_system(pdb.topology)

In [4]:
integrator = LangevinIntegrator(timestep=4.0 * unit.femtoseconds)
platform_name = 'CUDA'
platform = openmm.Platform.getPlatformByName(platform_name)
if platform_name in ['CUDA', 'OpenCL']:
    platform.setPropertyDefaultValue('Precision', 'mixed')
if platform_name in ['CUDA']:
    platform.setPropertyDefaultValue('DeterministicForces', 'true')

In [8]:
context = openmm.Context(system, integrator, platform)
context.setPositions(pdb.positions)
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -432.5253883410487 kJ/mol


In [9]:
openmm.LocalEnergyMinimizer.minimize(context)

In [10]:
potential_energy = context.getState(getEnergy=True).getPotentialEnergy()
print("POTENTIAL ENERGY: ", potential_energy)

POTENTIAL ENERGY:  -671.8551339108972 kJ/mol


In [11]:
pos = np.array(context.getState(getPositions=True).getPositions().value_in_unit_system(unit.md_unit_system))

In [12]:
app.PDBFile.writeFile(pdb.topology, context.getState(getPositions=True).getPositions(), open("N501Y_capped_peptide_old_minimized.pdb", "w"), keepIds=True)